In [60]:
import sys
sys.path.append('/home/jykim/work/xai_pyh/lipreading/models')

from lipreading.models.whisper_encoder import OpenAIWhisperEncoder

# /home/jykim/work/xai_pyh/lipreading/models/espnet2
# /home/jykim/work/xai_pyh/lipreading/models/whisper_encoder.py

model = OpenAIWhisperEncoder(whisper_model = 'base')

In [61]:
import torch
import torchaudio

audio_path = "/home/nas3/project/2022_XAI/Dataset/data_NIA/XAI_SEV_0919_relabel_healthy_autumn_crop/0/ori_02_75_1_0_1_006_1_sen1.wav"

waveform, sr = torchaudio.load(audio_path)
print(waveform.shape)
print(sr)

torch.Size([1, 73728])
16000


In [23]:
# import torch.nn as nn

# model2 = nn.Sequential(
#     nn.Conv1d(1,1, kernel_size=80, stride=4, padding=38,
#                                bias=False),
#     nn.AvgPool1d(kernel_size=21, padding=1)
# )
# output2 = model2(waveform)

# output2.shape

In [52]:
output = model(waveform, torch.tensor([waveform.size(1)]))
print(output[0], output[1] )
print(output[0].shape, output[1].shape)

tensor([[[-0.5790,  0.3248, -0.5361,  ..., -0.3087,  0.7005, -0.6046],
         [ 0.7679,  0.4173, -0.0404,  ..., -0.3458,  0.5922, -0.4441],
         [ 1.3745,  0.6994, -0.3357,  ..., -0.2838, -0.1222, -0.7942],
         ...,
         [ 0.6104,  0.5418, -0.1040,  ..., -0.0611,  0.1795,  0.1991],
         [ 0.7245, -0.3245, -0.2757,  ..., -0.2151, -0.1159, -0.3268],
         [ 1.1133, -0.0534,  0.3958,  ..., -0.3707, -0.8468,  0.3171]]],
       grad_fn=<NativeLayerNormBackward0>) tensor([115])
torch.Size([1, 115, 512]) torch.Size([1])


In [49]:
# npout = output[0].detach().numpy()

# # shape 출력

# npout.shape

In [93]:
import pandas as pd
import random

# df = pd.read_csv("/home/nas3/project/2022_XAI/Dataset/231011_autumn_split/data_autumn_train_1011.csv") # train
# df = pd.read_csv("/home/nas3/project/2022_XAI/Dataset/231011_autumn_split/data_autumn_valid_1011.csv") # val
df = pd.read_csv("/home/nas3/project/2022_XAI/Dataset/231011_autumn_split/data_autumn_test_1011.csv") # test

data_len = len(df)

l_audio, l_label = [], []

for n in range(data_len) :
    audio = df.iloc[n]['audio']
    label = df.iloc[n]['label']
    l_audio.append(audio)
    l_label.append(label)

In [94]:
import os
import numpy as np

# "/home/nas3/project/2022_XAI/Dataset/data_NIA/XAI_SEV_0919_relabel_healthy_autumn_crop/0/ori_02_75_1_0_1_006_1_sen5.wav"

for audio, label in zip(l_audio, l_label) :
    print(audio.split('/')[-1], label)
    waveform, sr = torchaudio.load(audio)
    print(str(waveform.size(1)) + " -> "+ f"waveform sec : {waveform.size(1)/sr}")
    output = model(waveform, torch.tensor([waveform.size(1)]))
    print(output[0].shape, output[1])
    print()
    print('-' * 80)
    
    new_audio_path = audio.replace('data_NIA', 'data_NIA_Whisper')
    save_path_base = '/'.join(new_audio_path.split('/')[:-1])
    if not os.path.exists(save_path_base) :
        os.makedirs(save_path_base)
    
    save_file_name = new_audio_path.split('/')[-1].split('.')[0] + ".npz"
    final_save_path = os.path.join(save_path_base, save_file_name)
    # print(final_save_path)
    # print()
    # print('-' * 80)
    
    tmp_dict = {}
    tmp_dict['real_len'] = waveform.size(1)
    tmp_dict['whisper_embeddings'] =  output[0].detach().numpy()
    
    np.savez(final_save_path, tmp_dict)
    

nia_HC0022_50_1_1_4_006_1_sen5.wav 1
36864 -> waveform sec : 2.304
torch.Size([1, 58, 512]) tensor([58])

--------------------------------------------------------------------------------
nia_HC0022_50_1_1_4_006_1_sen2.wav 1
86016 -> waveform sec : 5.376
torch.Size([1, 134, 512]) tensor([134])

--------------------------------------------------------------------------------
nia_HC0022_50_1_1_4_006_1_sen1.wav 1
40960 -> waveform sec : 2.56
torch.Size([1, 64, 512]) tensor([64])

--------------------------------------------------------------------------------
nia_HC0022_50_1_1_4_006_1_sen6.wav 1
194560 -> waveform sec : 12.16
torch.Size([1, 304, 512]) tensor([304])

--------------------------------------------------------------------------------
nia_HC0022_50_1_1_4_006_1_sen4.wav 1
139264 -> waveform sec : 8.704
torch.Size([1, 218, 512]) tensor([218])

--------------------------------------------------------------------------------
nia_HC0022_50_1_1_4_006_1_sen3.wav 1
167936 -> waveform se

In [119]:
import glob

path = "/home/nas3/project/2022_XAI/Dataset/data_NIA_Whisper/XAI_SEV_0919_relabel_healthy_autumn_crop/1/"

npz_list = glob.glob(path + "*.npz")

npz = np.load(npz_list[0], allow_pickle=True)

print(npz['arr_0'])

{'real_len': 77824, 'whisper_embeddings': array([[[-0.96094865, -0.7955127 , -1.6737105 , ..., -0.6208568 ,
          0.2703124 , -0.5297283 ],
        [-0.06997121, -0.4474726 , -0.98005944, ...,  0.47611156,
          0.23157066, -0.36118984],
        [-0.5561177 , -1.5977064 , -0.21000633, ...,  0.20782298,
         -1.0610212 , -0.36479396],
        ...,
        [ 0.38161403, -1.4501181 , -0.33285874, ...,  1.07823   ,
          0.9184294 ,  0.6198617 ],
        [-0.87235606,  0.83955914, -0.10770167, ..., -0.27564356,
          0.1783947 , -0.54218394],
        [ 0.50208867,  0.3222097 ,  0.85624665, ...,  1.3059149 ,
         -0.04363318, -1.073347  ]]], dtype=float32)}


In [120]:
import numpy as np

npz = np.load(npz_list[0], allow_pickle=True)
arr_0_data = npz['arr_0'].item()  # .item()을 사용하여 사전과 유사한 객체로 변환

real_len = arr_0_data['real_len']
whisper_embeddings = arr_0_data['whisper_embeddings']

print(real_len)
print(whisper_embeddings.shape)


77824
(1, 122, 512)


In [118]:
for i in range(len(npz_list)) :
    npz = np.load(npz_list[i], allow_pickle=True)
    arr_0_data = npz['arr_0'].item()  # .item()을 사용하여 사전과 유사한 객체로 변환

    real_len = arr_0_data['real_len']
    whisper_embeddings = arr_0_data['whisper_embeddings']

    print(real_len)
    print(whisper_embeddings.shape)
    print('-' * 80)

161294
(1, 252, 512)
--------------------------------------------------------------------------------
280214
(1, 438, 512)
--------------------------------------------------------------------------------
560427
(1, 876, 512)
--------------------------------------------------------------------------------
478413
(1, 748, 512)
--------------------------------------------------------------------------------
133956
(1, 209, 512)
--------------------------------------------------------------------------------
702583
(1, 1098, 512)
--------------------------------------------------------------------------------
50139
(1, 78, 512)
--------------------------------------------------------------------------------
92931
(1, 145, 512)
--------------------------------------------------------------------------------
177217
(1, 277, 512)
--------------------------------------------------------------------------------
160792
(1, 251, 512)
---------------------------------------------------------------

ResNet18과 비교

In [139]:
import torchvision
import torch.nn as nn

class ResNet_embeddings(nn.Module):
    def __init__(self, pretrained=False):
        super(ResNet_embeddings, self).__init__()
        self.model = torchvision.models.resnet18(pretrained=pretrained)
        self.features = nn.Sequential(*list(self.model.children())[:-2])
        
    def forward(self, x):
        x = self.features(x)
        # x = x.view(x.size(0), -1)
        return x

In [140]:
model1 = torchvision.models.resnet18(pretrained=False)
model2 = ResNet_embeddings()

/home/jykim/anaconda3/envs/pyh/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jykim/anaconda3/envs/pyh/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [148]:
import torch
import torchaudio

audio_path = "/home/nas3/project/2022_XAI/Dataset/data_NIA/XAI_SEV_0919_relabel_healthy_autumn_crop/0/ori_02_75_1_0_1_006_1_sen1.wav"

waveform, sr = torchaudio.load(audio_path)
print(waveform.shape)
# print(sr)

output = model(waveform, torch.tensor([waveform.size(1)]))
print(output[0].shape, output[1])

torch.Size([1, 73728])
torch.Size([1, 115, 512]) tensor([115])


In [149]:
SAMPLE_RATE = 16000
N_FFT = 400
N_MELS = 80
HOP_LENGTH = 160

from torchlibrosa.stft import Spectrogram, LogmelFilterBank

spectrogram_extractor = Spectrogram(n_fft=N_FFT, hop_length=HOP_LENGTH, win_length=N_FFT)
logmel_extractor = LogmelFilterBank(sr=SAMPLE_RATE, n_fft=N_FFT, n_mels=N_MELS, freeze_parameters=True)

spec_x = spectrogram_extractor(waveform)
print(spec_x.shape)
mel_spec_x = logmel_extractor(spec_x)
print(mel_spec_x.shape)
mel_spec_x = mel_spec_x.repeat(1,3,1,1)
print(mel_spec_x.shape)

torch.Size([1, 1, 461, 201])
torch.Size([1, 1, 461, 80])
torch.Size([1, 3, 461, 80])


In [156]:
def mel_filters(device, n_mels: int = N_MELS) -> torch.Tensor:
    assert n_mels == 80, f"Unsupported n_mels: {n_mels}"

    filters_path = "/home/jykim/work/xai_pyh/mel_filters.npz"
    with np.load(filters_path, allow_pickle=False) as f:
        return torch.from_numpy(f[f"mel_{n_mels}"])

def log_mel_spectrogram(audio):
    
    window = torch.hann_window(N_FFT)
    
    stft = torch.stft(
            audio, N_FFT, HOP_LENGTH, window=window, return_complex=True
        )
    
    magnitudes = stft[..., :-1].abs() ** 2
    
    filters = mel_filters(audio, N_MELS)
    mel_spec = filters @ magnitudes
    log_spec = torch.clamp(mel_spec, min=1e-10).log10()
    
    log_spec = torch.maximum(
            log_spec,
            log_spec.view(audio.size(0), -1).max(dim=-1)[0][:, None, None] - 8.0,
        )
    log_spec = (log_spec + 4.0) / 4.0
    
    return log_spec

In [164]:
try_mel_spec_x = log_mel_spectrogram(waveform)
try_mel_spec_x.shape
# try_mel_spec_x = OpenAIWhisperEncoder.log_mel_spectrogram(audio=waveform)
try_mel_spec_x.repeat(1,3,1,1).shape

torch.Size([1, 3, 80, 460])

In [161]:
res_output = model2(try_mel_spec_x.repeat(1,3,1,1))
res_output.shape

torch.Size([1, 512, 3, 15])

In [178]:
res_output_ori = model2(mel_spec_x.to(device))
res_output_ori.shape

torch.Size([1, 512, 15, 3])

In [173]:
from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
summary(model2.to(device), input_size = (3, 80, 460))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 40, 230]           9,408
            Conv2d-2          [-1, 64, 40, 230]           9,408
       BatchNorm2d-3          [-1, 64, 40, 230]             128
       BatchNorm2d-4          [-1, 64, 40, 230]             128
              ReLU-5          [-1, 64, 40, 230]               0
              ReLU-6          [-1, 64, 40, 230]               0
         MaxPool2d-7          [-1, 64, 20, 115]               0
         MaxPool2d-8          [-1, 64, 20, 115]               0
            Conv2d-9          [-1, 64, 20, 115]          36,864
           Conv2d-10          [-1, 64, 20, 115]          36,864
      BatchNorm2d-11          [-1, 64, 20, 115]             128
      BatchNorm2d-12          [-1, 64, 20, 115]             128
             ReLU-13          [-1, 64, 20, 115]               0
             ReLU-14          [-1, 64, 